In [3]:
!pip install influxdb-client

In [12]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
from urllib3.exceptions import NewConnectionError

INFLUX_URL = "http://influxdb2:8086"
INFLUX_TOKEN = "MyInitialAdminToken0=payosno"

print("--- Iniciando conexión a InfluxDB ---")

client = None
try:
    # 1. Inicializar el cliente
    client = influxdb_client.InfluxDBClient(
        url=INFLUX_URL,
        token=INFLUX_TOKEN,
        org="docs"
    )

    # 2. Verificar la conexión con el servidor
    print(f"Verificando estado de salud de InfluxDB en {INFLUX_URL}...")
    health = client.health()
    
    if health.status == "pass":
        print("[INFO] ¡Conexión exitosa!")
        print(f"[INFO]  Versión del servidor: {health.version}")
    else:
        print(f"[ERROR] Conexión fallida. Estado: {health.status}")
        print(f"[INFO] Mensaje: {health.message}")

except (InfluxDBError, NewConnectionError) as e:
    print("[ERROR] Error al conectar con InfluxDB:")
    print(f"   Detalle: {e}")
    
# finally:
#     # 3. Es buena práctica cerrar siempre el cliente
#     if client:
#         client.close()
#         print("--- Conexión cerrada ---")

--- Iniciando conexión a InfluxDB ---
Verificando estado de salud de InfluxDB en http://influxdb2:8086...
[INFO] ¡Conexión exitosa!
[INFO]  Versión del servidor: v2.7.12


In [13]:
from influxdb_client import Point
import datetime

tiempo_actual = datetime.datetime.utcnow().isoformat() + "Z"

p = Point("test") \
    .tag("ubicacion", "[1201]") \
    .tag("planta", 2) \
    .field("temperatura", 18.4) \
    .field("humedad", 60) \
    .time(tiempo_actual)

write_api = client.write_api(write_options=SYNCHRONOUS)

write_api.write(bucket="iot_data", org="docs", record=p)